In [3]:
from dotenv import load_dotenv 
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")
load_dotenv()

True

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, UnstructuredWordDocumentLoader

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size = 256,
  chunk_overlap = 20
)

loader = DirectoryLoader("temp", glob="*.docx", loader_cls=UnstructuredWordDocumentLoader)
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

In [6]:
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)

In [7]:
list(store.yield_keys())

[]

In [8]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'vtw-cached'

database = PineconeVectorStore.from_documents(document_list, cached_embedder, index_name=index_name)

c:\Users\김건홍\Desktop\back-workspace\vtw-ai-assistant-and-ocr\env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [9]:
list(store.yield_keys())[:5]

['text-embedding-3-large01981830-2530-5572-bce5-cabcc6a8c9ac',
 'text-embedding-3-large058e5a4a-6a3f-5698-9bad-78ff3765e6e7',
 'text-embedding-3-large072f2aff-f397-5ad7-9c87-703278d6fb85',
 'text-embedding-3-large075b9246-4fb6-584a-93f6-4b3d6ec86d13',
 'text-embedding-3-large08dff520-a698-542e-b2e7-60f35e762300']

In [10]:
%%time
database2 = PineconeVectorStore.from_documents(document_list, cached_embedder, index_name=index_name)

CPU times: total: 734 ms
Wall time: 5.36 s


In [ ]:
### 위는 모델 캐시

In [ ]:
### 여기서 부터 LLM 캐시